In [0]:

%pip install findspark
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


# LOAN DATASET #

In [0]:
#read in load dataset
loanDf = spark.table('default.loan')
oldcol = loanDf.columns
newcol = ['CustomerID', 'Age', 'Gender', 'Occupation', 'MaritalStatus', 'FamilySize', 'Income', 'Expenditure', 'UseFrequency', 'LoanCategory', 'LoanAmount', 'Overdue', 'DebtRecord', 'ReturnedCheck', 'DishonourOfBill']
for idx in range(len(oldcol)):
    loanDf = loanDf.withColumnRenamed(oldcol[idx], newcol[idx])

In [0]:
loanDf = loanDf.withColumn('LoanAmount', regexp_replace(loanDf['LoanAmount'],',','').cast('numeric'))
loanDf = loanDf.withColumn('DebtRecord', regexp_replace(loanDf['DebtRecord'],',','').cast('numeric'))

In [0]:
# Print the schema of the loan dataset
loanDf.printSchema()

root
 |-- CustomerID: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- FamilySize: long (nullable = true)
 |-- Income: long (nullable = true)
 |-- Expenditure: long (nullable = true)
 |-- UseFrequency: long (nullable = true)
 |-- LoanCategory: string (nullable = true)
 |-- LoanAmount: decimal(10,0) (nullable = true)
 |-- Overdue: long (nullable = true)
 |-- DebtRecord: decimal(10,0) (nullable = true)
 |-- ReturnedCheck: long (nullable = true)
 |-- DishonourOfBill: long (nullable = true)



In [0]:
# Print the first 5 lines of the loan dataset
loanDf.limit(5).show()

+----------+---+------+------------+-------------+----------+------+-----------+------------+------------+----------+-------+----------+-------------+---------------+
|CustomerID|Age|Gender|  Occupation|MaritalStatus|FamilySize|Income|Expenditure|UseFrequency|LoanCategory|LoanAmount|Overdue|DebtRecord|ReturnedCheck|DishonourOfBill|
+----------+---+------+------------+-------------+----------+------+-----------+------------+------------+----------+-------+----------+-------------+---------------+
|   IB14001| 30|  MALE|BANK MANAGER|       SINGLE|         4| 50000|      22199|           6|     HOUSING|   1000000|      5|     42898|            6|              9|
|   IB14008| 44|  MALE|   PROFESSOR|      MARRIED|         6| 51000|      19999|           4|    SHOPPING|     50000|      3|     33999|            1|              5|
|   IB14012| 30|FEMALE|     DENTIST|       SINGLE|         3| 58450|      27675|           5|  TRAVELLING|     75000|      6|     20876|            3|              1

In [0]:
# Print the number of columns in the loan dataset
len(loanDf.columns)

15

In [0]:
# Print the number of rows in the loan dataset
loanDf.count()

500

In [0]:
# Print the count of distinct records in the loan dataset
loanDf.distinct().count()

500

In [0]:
# find the number of loans in each category
loanDf.groupBy('LoanCategory').count().show()

+------------------+-----+
|      LoanCategory|count|
+------------------+-----+
|           HOUSING|   67|
|        TRAVELLING|   53|
|       BOOK STORES|    7|
|       AGRICULTURE|   12|
|         GOLD LOAN|   77|
|  EDUCATIONAL LOAN|   20|
|        AUTOMOBILE|   60|
|          BUSINESS|   24|
|COMPUTER SOFTWARES|   35|
|           DINNING|   14|
|          SHOPPING|   35|
|       RESTAURANTS|   41|
|       ELECTRONICS|   14|
|          BUILDING|    7|
|        RESTAURANT|   20|
|   HOME APPLIANCES|   14|
+------------------+-----+



In [0]:
# find the number of people who have taken more than 1 lack loan
loanDf.where(col('LoanAmount') > 1000000).select('CustomerID').distinct().count()

130

In [0]:
# find the number of people with income greater than 60000 rupees
loanDf.where(col('Income')>60000).count()

198

In [0]:
# find the number of people with 2 or more returned cheques and income less than 50000
loanDf.where((col('ReturnedCheck') >= 2)&(loanDf.Income < 50000)).count()

137

In [0]:
# find the number of people with 2 or more returned cheques and are single
loanDf.where((col('ReturnedCheck') >= 2)&(col('MaritalStatus') == 'SINGLE')).count()

111

In [0]:
# find the  number of people with expenditure over 50000 a month 
loanDf.where(col('Expenditure') > 50000).show()

+----------+---+------+---------------+-------------+----------+------+-----------+------------+------------+----------+-------+----------+-------------+---------------+
|CustomerID|Age|Gender|     Occupation|MaritalStatus|FamilySize|Income|Expenditure|UseFrequency|LoanCategory|LoanAmount|Overdue|DebtRecord|ReturnedCheck|DishonourOfBill|
+----------+---+------+---------------+-------------+----------+------+-----------+------------+------------+----------+-------+----------+-------------+---------------+
|   IB14158| 54|  MALE|AIRPORT OFFICER|      MARRIED|         6| 80000|      62541|           2|  AUTOMOBILE|   2045789|      1|     16599|            2|              3|
|   IB14176| 54|  MALE|AIRPORT OFFICER|      MARRIED|         6| 80000|      62541|           2|     HOUSING|   2045789|      1|     16599|            2|              3|
|   IB14204| 54|  MALE|AIRPORT OFFICER|      MARRIED|         6| 81000|      62541|           2|     DINNING|   2045789|      1|     16599|           

# CREDIT CARD DATASET #

In [0]:
# Load the credit card dataset
credit_cardDf = spark.table('default.credit_card')

In [0]:
# Print the schema of the credit card dataset
credit_cardDf.printSchema()

root
 |-- RowNumber: long (nullable = true)
 |-- CustomerId: long (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: long (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Tenure: long (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: long (nullable = true)
 |-- IsActiveMember: long (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: long (nullable = true)



In [0]:
# Print the number of columns in the credit card dataset
len(credit_cardDf.columns)

13

In [0]:
# Print the number of rows in the credit card dataset
credit_cardDf.count()

10000

In [0]:
# Print the number of distinct records in the credit card dataset
credit_cardDf.distinct().count()

10000

In [0]:
# Print the first 5 rows in the credit card dataset
credit_cardDf.limit(5).show()

+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|      0.0|            1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        3|  15619304|    Onio|        502|   France|Female| 42|     8| 159660.8|            3|             0|      113931.57|     1|
|        4|  15701354|    Boni|        699|   France|Female| 39|     1|      0.0|            2|             0|       93826.63|     0|
|        5|  15737888|Mitchell|        850|    Spain|Female| 4

In [0]:
# Find the number of members who are elgible for credit card
credit_cardDf.where(col('CreditScore') > 700).count()

3116

In [0]:
# Find the number of members who are  elgible and active in the bank
credit_cardDf.where((col('CreditScore') > 700) & (col('IsActiveMember') == 1)).count()

1637

In [0]:
# Find the credit card users in Spain 
credit_cardDf.where(col('Geography') == 'Spain').show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+--------------+---------------+------+
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|             1|      112542.58|     0|
|        5|  15737888| Mitchell|        850|    Spain|Female| 43|     2|125510.82|            1|             1|        79084.1|     0|
|        6|  15574012|      Chu|        645|    Spain|  Male| 44|     8|113755.78|            2|             0|      149756.71|     1|
|       12|  15737173|  Andrews|        497|    Spain|  Male| 24|     3|      0.0|            2|             0|       76390.01|     0|
|       15|  15600882|    Scott|        635|    Spain|F

In [0]:
# Find the credit card users with Estiamted Salary greater than 100000 and have exited the card
credit_cardDf.where((col('EstimatedSalary') > 100000) & (col('Exited') == 1)).count()

1044

In [0]:
# Find the credit card users with Estiamted Salary less than 100000 and have more than 1 products
credit_cardDf.where((col('EstimatedSalary') < 100000) & (col('NumOfProducts') > 1)).count()

2432

# TRANSACTION DATASET #

In [0]:
# Load the transacton dataset
txnDf = spark.table('default.txn')
oldcol = txnDf.columns
newcol = ['AccountNo', 'TransactionDetails', 'ValueDate', 'WithdrawalAmt', 'DepositAmt', 'BalanceAmt']

for idx in range(len(oldcol)):
    txnDf = txnDf.withColumnRenamed(oldcol[idx], newcol[idx])

In [0]:
# Print the schema of the transacton dataset
txnDf.printSchema()

root
 |-- AccountNo: string (nullable = true)
 |-- TransactionDetails: string (nullable = true)
 |-- ValueDate: string (nullable = true)
 |-- WithdrawalAmt: double (nullable = true)
 |-- DepositAmt: double (nullable = true)
 |-- BalanceAmt: double (nullable = true)



In [0]:
#COUNT OF TRANSACTION ON EVERY ACCOUNT
txnDf.groupBy('AccountNo').count().show()

+-------------+-----+
|    AccountNo|count|
+-------------+-----+
|409000438611'| 4588|
|     1196711'|10536|
|     1196428'|48779|
|409000493210'| 6014|
|409000611074'| 1093|
|409000425051'|  802|
|409000405747'|   51|
|409000493201'| 1044|
|409000438620'|13454|
|409000362497'|29840|
+-------------+-----+



In [0]:
# Find the Maximum withdrawal amount for each account
txnDf.groupBy('AccountNo').max('WithdrawalAmt').orderBy('max(WithdrawalAmt)', ascending=False).show()

+-------------+------------------+
|    AccountNo|max(WithdrawalAmt)|
+-------------+------------------+
|     1196711'|     4.594475464E8|
|409000438620'|             4.0E8|
|409000425051'|            3.54E8|
|409000438611'|             2.4E8|
|409000405747'|             1.7E8|
|     1196428'|             1.5E8|
|409000362497'|     1.413662392E8|
|409000493210'|             1.5E7|
|409000493201'|         2500000.0|
|409000611074'|          912000.0|
+-------------+------------------+



In [0]:
#MINIMUM WITHDRAWAL AMOUNT OF AN ACCOUNT
txnDf.groupBy('AccountNo').min('WithdrawalAmt').orderBy('min(WithdrawalAmt)').show()

+-------------+------------------+
|    AccountNo|min(WithdrawalAmt)|
+-------------+------------------+
|409000493210'|              0.01|
|409000438611'|               0.2|
|     1196428'|              0.25|
|     1196711'|              0.25|
|409000438620'|              0.34|
|409000362497'|              0.97|
|409000425051'|              1.25|
|409000493201'|               2.1|
|409000405747'|              21.0|
|409000611074'|             120.0|
+-------------+------------------+



In [0]:
#MAXIMUM DEPOSIT AMOUNT OF AN ACCOUNT
txnDf.groupBy('AccountNo').max('DepositAmt').orderBy('max(DepositAmt)', ascending=False).show()

+-------------+---------------+
|    AccountNo|max(DepositAmt)|
+-------------+---------------+
|409000438620'|        5.448E8|
|     1196711'|          5.0E8|
|     1196428'|  2.119594422E8|
|409000405747'|        2.021E8|
|409000362497'|          2.0E8|
|409000438611'|       1.7025E8|
|409000493210'|          1.5E7|
|409000425051'|          1.5E7|
|409000611074'|      3000000.0|
|409000493201'|      1000000.0|
+-------------+---------------+



In [0]:
#MINIMUM DEPOSIT AMOUNT OF AN ACCOUNT
txnDf.groupBy('AccountNo').min('DepositAmt').orderBy('min(DepositAmt)').show()

+-------------+---------------+
|    AccountNo|min(DepositAmt)|
+-------------+---------------+
|409000493210'|           0.01|
|409000438611'|           0.03|
|409000362497'|           0.03|
|409000438620'|           0.07|
|409000493201'|            0.9|
|409000425051'|            1.0|
|     1196428'|            1.0|
|     1196711'|           1.01|
|409000405747'|          500.0|
|409000611074'|         1320.0|
+-------------+---------------+



In [0]:
#sum of balance in every bank account
txnDf.groupBy('AccountNo').sum('BalanceAmt').show()

+-------------+--------------------+
|    AccountNo|     sum(BalanceAmt)|
+-------------+--------------------+
|409000438611'|-2.49486577068339...|
|     1196711'|-1.60476498101275E13|
|     1196428'| -8.1418498130721E13|
|409000493210'|-3.27584952132095...|
|409000611074'|       1.615533622E9|
|409000425051'|-3.77211841164998...|
|409000405747'|-2.43108047067000...|
|409000493201'|1.0420831829499985E9|
|409000438620'|-7.12291867951358...|
|409000362497'| -5.2860004792808E13|
+-------------+--------------------+



In [0]:
#Number of transaction on each date
txnDf.groupBy('ValueDate').count().orderBy('count', ascending=False).show()

+---------+-----+
|ValueDate|count|
+---------+-----+
|27-Jul-17|  567|
|13-Aug-18|  463|
| 8-Nov-17|  402|
| 7-Oct-17|  382|
|10-Jul-18|  374|
|12-Dec-17|  367|
|12-Sep-18|  365|
| 9-Aug-18|  360|
|19-Sep-17|  358|
|16-Mar-17|  353|
|10-Sep-18|  344|
|14-Jul-17|  333|
| 7-Mar-18|  319|
|11-Oct-18|  303|
|22-Aug-17|  301|
| 9-Jan-18|  299|
| 9-Oct-18|  297|
|20-Apr-18|  296|
| 9-Jul-18|  292|
| 7-Apr-18|  291|
+---------+-----+
only showing top 20 rows



In [0]:
#List of customers with withdrawal amount more than 1 lakh (100000)
txnDf.select(['AccountNo', 'TransactionDetails', 'WithdrawalAmt']).where(col('WithdrawalAmt') > 100000).show()

+-------------+--------------------+-------------+
|    AccountNo|  TransactionDetails|WithdrawalAmt|
+-------------+--------------------+-------------+
|409000611074'|INDO GIBL Indiafo...|     133900.0|
|409000611074'|INDO GIBL Indiafo...|     195800.0|
|409000611074'|INDO GIBL Indiafo...|     143800.0|
|409000611074'|INDO GIBL Indiafo...|     331650.0|
|409000611074'|INDO GIBL Indiafo...|     129000.0|
|409000611074'|INDO GIBL Indiafo...|     230013.0|
|409000611074'|INDO GIBL Indiafo...|     367900.0|
|409000611074'|INDO GIBL Indiafo...|     108000.0|
|409000611074'|INDO GIBL Indiafo...|     141000.0|
|409000611074'|INDO GIBL Indiafo...|     206000.0|
|409000611074'|INDO GIBL Indiafo...|     242300.0|
|409000611074'|INDO GIBL Indiafo...|     113250.0|
|409000611074'|INDO GIBL Indiafo...|     206900.0|
|409000611074'|INDO GIBL Indiafo...|     276000.0|
|409000611074'|INDO GIBL Indiafo...|     171000.0|
|409000611074'|INDO GIBL Indiafo...|     189800.0|
|409000611074'|INDO GIBL Indiaf